prototype_ngo_scraping_parallelized

# Prototype NGO Scraping: Scraping Information for Multiple NGOs
The objective of this notebook is to play around with the NGO website scraping to in order to find a good way to scrape the list of NGO's. 

## imports

In [1]:
#--------DISPLAY SETTINGS FOR THE NOTEBOOK--------
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' #to see all outputs of a cell, as opposed to only the last one
from IPython.display import display, HTML # Alejandro-Note: Changed from IPython.display
display(HTML("<style>.container { width:100% !important; }</style>")) #make the notebook take up more screen space (default is 60%)

#-------FOR WORKING WITH DATA IN A DATAFRAME--------

import pandas as pd #To store scraped data

#-------SCRAPING SPECIFIC MODULES--------
import requests #to conduct different forms of HTTP requests
import html5lib #to construct tree structure of HTML data
from bs4 import BeautifulSoup as soup #to parse the html data obtained from the scrape
import time # to add wait time, to keep the website from kicking us out and also to let the page load before grabbing data
from selenium import webdriver #to automate the navigating within the browser
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select #to select the features we want on the website via the scraper
from selenium.webdriver.support.ui import WebDriverWait #again, to add wait times more 'implicitly'
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options #to use properties of the chrome webbrowser
from selenium.webdriver.remote.command import Command # Use to check whether the web driver is active

import random
from joblib import Parallel, delayed # for parallelizing
from tqdm import tqdm # this provides a visual progress tracker for loops

import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, wait
import threading, queue
from itertools import repeat
from concurrent import futures
from multiprocessing import Pool  # This is a CPU-based Pool
from multiprocessing import cpu_count

import re # Use regular expressions


In [2]:
root_directory = "D:/GitHub/expert-forecasting/"
store_directory = root_directory + "raw_data/3_get_policymaker_list/ngo_india_scraping/"

print(root_directory)
print(store_directory)

D:/GitHub/expert-forecasting/
D:/GitHub/expert-forecasting/raw_data/3_get_policymaker_list/ngo_india_scraping/


### set-up

#### specifying the url to be scraped

In [3]:
#base_url = 'https://ngodarpan.gov.in/index.php/home/sectorwise'
base_url = 'https://ngodarpan.gov.in/index.php/home/statewise'

#### setting up selenium to use chrome as the browser

In [4]:
options = Options()
options.headless = True # True hides the navigating of the browser by the scraper, False shows you the tab/window opening and stuff getting clicked
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\asanc\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [5]:
#--- This function creates a sector-specific URL
def get_sector_urls(base_url): 
    driver.get(base_url)
    sector_urls = [url.get_attribute('href') for url in driver.find_elements_by_class_name('bluelink11px')]
    sector_urls = [url+'?per_page=100' for url in sector_urls]
    return sector_urls

In [6]:
#--- This function captures the number of pages per sector
def get_num_of_pages_for_sector(sector_num,driver): 
    sector_url = sector_urls[sector_num]
    driver.get(sector_url)
    list_buttonlast = driver.find_elements_by_partial_link_text('Last')
    # If there are more than 10 pages, extract the link from the "last page".
    # Otherwise count how many page buttons there are (excluding the "previous" and "next" buttons)
    if len(list_buttonlast) > 0:
        driver.find_elements_by_partial_link_text('Last')[0].click()
        last_page_url = driver.current_url
        total_pages = int(last_page_url.split('?')[0].rsplit('/',1)[1])
    else:
        list_page_nums = driver.find_elements_by_xpath("//ul[@class='pagination']//li")
        if len(list_page_nums) == 0:
            total_pages = 1
        else:
            total_pages = len(list_page_nums)-2
    return total_pages

In [7]:
# get_num_of_pages_for_sector(sector_num = 2,driver = driver)


In [8]:
#--- This function extracts the page url for a specific sector + page number
def get_page_url(sector_url, page_num):
    page_url = sector_url.split('?',1)[0][:-1]+f'{page_num}?per_page=100'
    return page_url

In [9]:
#--- This function takes a list of NGO's per page, clicls through them sequentially,
#--- and extracts information from pop-up.
def scrape_a_single_page(sector_num, page_num): 
    
    # The following code selects a web_driver from the dictionary.
    # Both "drivers_dict" and "max_workers are defined in the environment."
    #driver = drivers_dict[page_num % max_workers_num]
    
    try:
        thread_name= threading.current_thread().name
        #print(threading.current_thread().name)
        #sometime we are going to have different thread name in each iteration so a little regex might help
        # This code avoids leaving additional threads running whenever "ThreadPoolExecutor" is run
        thread_name = re.sub("ThreadPoolExecutor-(\d*)_(\d*)", r"ThreadPoolExecutor-0_\2", thread_name)
        #print(f"re.sub -> {thread_name}")
        driver = drivers_dict[thread_name]
    except KeyError:
        drivers_dict[threading.current_thread().name] = webdriver.Chrome(ChromeDriverManager().install(), options=options)
        driver = drivers_dict[threading.current_thread().name]    
    
    sector_url = sector_urls[sector_num]
    page_url   = get_page_url(sector_url, page_num)
    driver.get(page_url)
    time.sleep(2)
    ngos_on_page = driver.find_elements_by_xpath("//a[contains(@onclick,'show_ngo_info')]")
    page_df = pd.DataFrame()
    for i in range(0, len(ngos_on_page)): 
    #for i in range(0, 3):
        print(f'scraping NGO number {i+1}')
        ngo = ngos_on_page[i]
        time.sleep(2)
        ngo.click()
#         print('Opened NGO info box')
#         print('Extracting Details')
        time.sleep(2)
        name = driver.find_element_by_id('ngo_name_title').get_attribute('innerHTML')
        uid = driver.find_element_by_id('UniqueID').get_attribute('innerHTML')
        reg_with = driver.find_element_by_id('reg_with').get_attribute('innerHTML')
        ngo_type = driver.find_element_by_id('ngo_type').get_attribute('innerHTML')
        ngo_regno = driver.find_element_by_id('ngo_regno').get_attribute('innerHTML')
        rc_upload = driver.find_element_by_id('rc_upload').get_attribute('innerHTML')
        pc_upload = driver.find_element_by_id('pc_upload').get_attribute('innerHTML')
        act_name = driver.find_element_by_id('ngo_act_name').get_attribute('innerHTML')
        city_reg = driver.find_element_by_id('ngo_city_p').get_attribute('innerHTML')
        state_reg = driver.find_element_by_id('ngo_state_p').get_attribute('innerHTML')
        reg_date = driver.find_element_by_id('ngo_reg_date').get_attribute('innerHTML')
        key_issues = driver.find_element_by_id('key_issues').get_attribute('innerHTML')
        operational_states = driver.find_element_by_id('operational_states').get_attribute('innerHTML')
        operational_districts = driver.find_element_by_id('operational_district').get_attribute('innerHTML')
        fcra_details = driver.find_element_by_id('FCRA_details').get_attribute('innerHTML')
        fcra_regno = driver.find_element_by_id('FCRA_reg_no').get_attribute('innerHTML')
        details_achievement = driver.find_element_by_id('activities_achieve').get_attribute('innerHTML')
        contact_address = driver.find_element_by_id('address').get_attribute('innerHTML')
        contact_city = driver.find_element_by_id('city').get_attribute('innerHTML')
        contact_state = driver.find_element_by_id('state_p_ngo').get_attribute('innerHTML')
        contact_telephone = driver.find_element_by_id('phone_n').get_attribute('innerHTML')
        contact_mobile = driver.find_element_by_id('mobile_n').get_attribute('innerHTML')
        contact_website = driver.find_element_by_id('ngo_web_url').get_attribute('innerText')
        contact_email = driver.find_element_by_id('email_n').get_attribute('innerHTML')
#         print('Extracting details from members table...')
        members_table = driver.find_element_by_id('member_table')
        member_names =  [i.get_attribute('innerHTML') for i in members_table.find_elements_by_xpath('.//tr//td')[::4]]
        member_designations = [i.get_attribute('innerHTML') for i in members_table.find_elements_by_xpath('.//tr//td')[1::4]]
        member_pan = [i.get_attribute('innerHTML') for i in members_table.find_elements_by_xpath('.//tr//td')[2::4]]
        member_aadhar = [i.get_attribute('innerHTML') for i in members_table.find_elements_by_xpath('.//tr//td')[3::4]]
        member_name_designation_dict = dict(zip(member_names, member_designations))
        member_name_pan_dict = dict(zip(member_names, member_pan))
        member_name_aadhar_dict = dict(zip(member_names, member_aadhar))
#         print('Extracting details from Source of Funds table...')
        sof_table = driver.find_element_by_id('source_table')
        dept_name = [i.get_attribute('innerHTML') for i in sof_table.find_elements_by_xpath('.//tr//td')[::5]]
        source = [i.get_attribute('innerHTML') for i in sof_table.find_elements_by_xpath('.//tr//td')[1::5]]
        financial_year = [i.get_attribute('innerHTML') for i in sof_table.find_elements_by_xpath('.//tr//td')[2::5]]
        amount_sanctioned =[i.get_attribute('innerHTML') for i in sof_table.find_elements_by_xpath('.//tr//td')[3::5]]
        purpose = [i.get_attribute('innerHTML') for i in sof_table.find_elements_by_xpath('.//tr//td')[4::5]]
        year_amount_dict = dict(zip(financial_year, amount_sanctioned))
        year_dept_dict = dict(zip(financial_year, dept_name))
        year_source_dict = dict(zip(financial_year, source))
        year_purpose_dict = dict(zip(financial_year, purpose))
#         print('Storing extracted info into a dataframe')
        df = pd.DataFrame()
        df['ngo_name'] = [name]
        df['unique_id'] = uid
        df['registered_with'] = reg_with
        df['type_of_ngo'] = ngo_type
        df['registration_number'] = ngo_regno
        df['copy_of_registration_certificate'] = rc_upload
        df['copy_of_pan_card'] = pc_upload
        df['act_name'] = act_name
        df['city_of_registration'] = city_reg
        df['state_of_registration'] = state_reg
        df['registration_date'] = reg_date
        df['key_issues'] = key_issues
        df['operational_areas_states'] = operational_states
        df['operational_areas_districts'] = operational_districts
        df['FCRA_details'] = fcra_details
        df['FCRA_registration_num'] = fcra_regno
        df['details_of_achievement'] = details_achievement
        df['contact_details_address'] = contact_address
        df['contact_details_city'] = contact_city
        df['contact_details_state'] = contact_state
        df['contact_details_telephone'] = contact_telephone
        df['contact_details_website'] = contact_website
        df['contact_details_email'] = contact_email
        df['members_names_designations'] = [member_name_designation_dict]
        df['members_names_pan_availability'] = [member_name_pan_dict]
        df['members_names_aadhar_availability'] = [member_name_aadhar_dict]
        df['source_of_funds_amount_sanctioned'] = [year_amount_dict]
        df['source_of_funds_department_name'] = [year_dept_dict]
        df['source_of_funds_source'] = [year_source_dict]
        df['source_of_funds_purpose'] = [year_purpose_dict]
        df['sector_number'] = sector_num
        df['ngo_num'] = i
        df['page_num'] = page_num
        df['page_num_total'] = len(ngos_on_page)
#         print('Appending to dataframe for all NGOs...')
        page_df = pd.concat([page_df,df]) # Alejandro-note: Use pandas.concat instead. Old method deprecated. page_df.append(df) 
        page_df.to_csv(f'{store_directory}/sectorno_{sector_num}_pageno_{page_num}_ngo_scraping.csv', index=False)
#         print('Closing pop-up window')
        close_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ngo_info_modal > div.modal-dialog.modal-lg > div > div.modal-header > button')))
        time.sleep(2)
        close_button.click()
    # drivers_dict[threading.current_thread().name].quit()
    return page_df

In [10]:
def scrape_a_sector(sector_num): 
    sector_url = sector_urls[sector_num]
    total_pages = get_num_of_pages_for_sector(sector_num)
    sector_df = pd.DataFrame()
    unsuccessful_pages= pd.DataFrame()
    for page in tqdm(range(1, total_pages+1)):
        print(f'Scraping page number {page}')
        page_url = get_page_url(sector_url, page)
        print(f'Scraping {page_url}')
        try: 
            page_df = scrape_a_single_page(sector_num, page_num= page)
            sector_df = pd.concat([sector_df,page_df]) # Alejandro-note: Use pandas.concat instead. Old method deprecated. sector_df.append(page_df)
            print(f'Page number {page} of {total_pages} finished')
        except Exception as e: 
            print(f'Exception {e} occurred for sector number {0} and page number {page}' )
            temp_df = pd.DataFrame()
            temp_df['sector_num'] = [sector_num]
            temp_df['page_url'] =page_url
            unsuccessful_pages = unsuccessful_pages.append(temp_df)
            unsuccessful_pages.to_csv(f'{store_directory}/unsuccessfully_scraped_pages.csv', index=False)
            continue
    return sector_df

In [11]:
# Scrape the list of ngos in each page
def scrape_list_singleplage(sector_num,page_num,driver): 
    
    sector_url = sector_urls[sector_num]
    page_url   = get_page_url(sector_url, page_num)
    driver.get(page_url)
    time.sleep(2)
    ngos_on_page = driver.find_elements_by_xpath("//a[contains(@onclick,'show_ngo_info')]")
    page_df = pd.DataFrame()

    # This command extracts the table of ngos + some extraneous information
    # The second row subsets the first 500 elements (100 organizations x 5 attributes)
    ngolist_table = driver.find_element_by_xpath(".//table[contains(@class,'table table-striped table-bordered table-hover Tax')]");
    raw_table = [i.get_attribute('innerHTML') for i in ngolist_table.find_elements_by_xpath('//tbody//tr//td')][0:500]

    ngo_numinpage    = raw_table[0::5]
    ngo_hyperlink    = raw_table[1::5]
    ngo_registration = raw_table[2::5]
    ngo_address      = raw_table[3::5]
    ngo_sectors      = raw_table[4::5]
    ngo_names = [i.split("<")[1].split(">")[1] for i in ngo_hyperlink]

    page_df['sector_num']    = [sector_num for i in range(0,len(ngo_numinpage))]
    page_df['page_num']      = [page_num for i in range(0,len(ngo_numinpage))]
    page_df['index_in_page'] = [i for i in range(0,len(ngo_numinpage))]
    page_df['ngo_name']     = ngo_names
    page_df['ngo_hyperlink'] = ngo_hyperlink
    page_df['ngo_registration'] = ngo_registration
    page_df['ngo_address']  = ngo_address
    page_df['ngo_sectors']  = ngo_sectors

    # page_df.to_csv(f'{store_directory}/list_sectorno_{sector_num}_page{page_num}_ngo_scraping.csv', index=False)    
    
    return(page_df)

In [12]:
# page_num = 1
# try:
#     thread_name= threading.current_thread().name
#     #print(threading.current_thread().name)
#     #sometime we are going to have different thread name in each iteration so a little regex might help
#     # This code avoids leaving additional threads running whenever "ThreadPoolExecutor" is run
#     thread_name = re.sub("ThreadPoolExecutor-(\d*)_(\d*)", r"ThreadPoolExecutor-0_\2", thread_name)
#     #print(f"re.sub -> {thread_name}")
#     driver = drivers_dict[thread_name]
# except KeyError:
#     drivers_dict[threading.current_thread().name] = webdriver.Chrome(ChromeDriverManager().install(), options=options)
#     driver = drivers_dict[threading.current_thread().name]  

# sector_url = sector_urls[sector_num]
# page_url   = get_page_url(sector_url, page_num)
# driver.get(page_url)
# time.sleep(2)
# ngos_on_page = driver.find_elements_by_xpath("//a[contains(@onclick,'show_ngo_info')]")
# page_df = pd.DataFrame()

# # This command extracts the table of ngos + some extraneous information
# # The second row subsets the first 500 elements (100 organizations x 5 attributes)
# ngolist_table = driver.find_element_by_xpath(".//table[contains(@class,'table table-striped table-bordered table-hover Tax')]");
# raw_table = [i.get_attribute('innerHTML') for i in ngolist_table.find_elements_by_xpath('//tbody//tr//td')][0:500]

# ngo_numinpage    = raw_table[0::5]
# ngo_hyperlink    = raw_table[1::5]
# ngo_registration = raw_table[2::5]
# ngo_address      = raw_table[3::5]
# ngo_sectors      = raw_table[4::5]
# ngo_names = [i.split("<")[1].split(">")[1] for i in ngo_hyperlink]

# page_df['numinpage'] = ngo_numinpage
# page_df['ngo_names'] = [sector_num for i in range(0,len(ngo_numinpage))]
# page_df['ngo_names']   = ngo_names
# page_df['ngo_hyperlink'] = ngo_hyperlink
# page_df['ngo_registration'] = ngo_registration
# page_df['ngo_address'] = ngo_address
# page_df['ngo_sectors'] = ngo_sectors

# # page_df

# page_df.to_csv(f'{store_directory}/list_sectorno_{sector_num}_page{page_num}_ngo_scraping.csv', index=False)    

In [13]:
def scrape_list_ngo_sector(sector_num): 
    print(f'Scraping sector number {sector_num}')  
    sector_url = sector_urls[sector_num]
    # print(f'Scraping sector number {sector_url}')
    #total_pages = total_pages_list[sector_num]
    #print(f'Scraping sector number {total_pages}')    
    #print(f'Initiating Web Driver') 
    try:
        thread_name= threading.current_thread().name
        #print(threading.current_thread().name)
        #sometime we are going to have different thread name in each iteration so a little regex might help
        # This code avoids leaving additional threads running whenever "ThreadPoolExecutor" is run
        thread_name = re.sub("ThreadPoolExecutor-(\d*)_(\d*)", r"ThreadPoolExecutor-0_\2", thread_name)
        #print(f"re.sub -> {thread_name}")
        driver = drivers_dict[thread_name]
        print(f"{thread_name}")
    except KeyError:
        drivers_dict[threading.current_thread().name] = webdriver.Chrome(ChromeDriverManager().install(), options=options)
        driver = drivers_dict[threading.current_thread().name] 
    
    print(f'Obtaining total number of pages')    
    total_pages = get_num_of_pages_for_sector(sector_num,driver)
    
    time.sleep(4)
    print(f'Initiating Scraping of NGO lists')
    
    sector_df = pd.DataFrame()
    #unsuccessful_pages= pd.DataFrame()
    for page in tqdm(range(1, total_pages+1)):
        print(f'Scraping Sector {sector_num}, page number {page}')
        page_url = get_page_url(sector_url, page)
        #print(f'Scraping {page_url}')
        page_df = scrape_list_singleplage(sector_num, page_num= page,driver=driver)
        sector_df = pd.concat([sector_df,page_df])

    sector_df.to_csv(f'{store_directory}/list_sectorno_{sector_num}_ngo_scraping.csv', index=False)    
        
    return sector_df

In [14]:
print(f"Obtaining URLs")
sector_urls = get_sector_urls(base_url)
#total_pages_list = [get_num_of_pages_for_sector(sector_num) for sector_num in range(0,len(sector_urls))]
drivers_dict={}

Obtaining URLs


In [15]:
# sector_urls[9]
# get_num_of_pages_for_sector(1,driver)

# scrape_list_ngo_sector(1)

In [16]:
# print(f"Inititate Multithreading for Extracting List of NGOs")
# max_workers_num = 10

# with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers_num) as executor:
#     result = executor.map(lambda sector_num_parallel: scrape_list_ngo_sector(sector_num_parallel), range(0,len(sector_urls)))
    

In [17]:
# sector_urls[1]

In [18]:
def scrape_multiple_pages(sector_num, start_page_num, end_page_num): 
    page_list = list(range(start_page_num, end_page_num+1))
    main_df = pd.DataFrame()
    unsuccessful_pages = pd.DataFrame()
    sector_url = sector_urls[sector_num]
    # The "tqdm" command helps us keep track of progress.
    for page in tqdm(page_list): 
        print(f'Scraping page number {page}')
        page_url = get_page_url(sector_url, page)
        print(f'Scraping {page_url}')
        try: 
            page_df = scrape_a_single_page(sector_num, page_num= page)
            main_df = pd.concat([main_df,page_df]) # Alejandro-note: Use pandas.concat instead. Old method deprecated. main_df.append(page_df)
        except Exception as e: 
            print(f'Exception {e} occurred for sector number {0} and page number {page}' )
            temp_df = pd.DataFrame()
            temp_df['sector_num'] = [sector_num]
            temp_df['page_url'] =page_url
            unsuccessful_pages = pd.concat([unsuccessful_pages,temp_df]) # Alejandro-note: Use pandas.concat instead. Old method deprecated. unsuccessful_pages.append(temp_df) 
            unsuccessful_pages.to_csv('./unsuccessfully_scraped_pages_sector{sector_num}_page{page}.csv', index=False)
            continue

In [19]:
# Command to compute the cumulative count of a dataset

# list_allngos = pd.DataFrame()

# #for sector_num in range(0,len(sector_urls)):
# for sector_num in range(1,3):
#     sector_df = pd.read_csv(f'{store_directory}/list_sectorno_{sector_num}_ngo_scraping.csv')
#     list_allngos = pd.concat([list_allngos,sector_df])

# # Create dianostic variables for possible duplicates in the list.
# list_allngos['duplicates_index'] = list_allngos.groupby(["ngo_hyperlink","ngo_registration"]).cumcount()+1
# list_allngos['total_duplicates'] = list_allngos.groupby(["ngo_hyperlink","ngo_registration"])["ngo_name"].transform('size')

# list_allngos.to_csv(f'{store_directory}/list_all_ngo_scraping.csv', index=False)    
        
# list_allngos


#

In [20]:
#sector_urls = get_sector_urls(base_url)

In [21]:
# ## This code creates initiates a list of web drivers used for multi-threading
# ## Intuitively, this is like opening multiple tabs on Chrome web browser.

# max_workers_num = 10
# drivers_dict={}

# for i in range(0,max_workers_num-1):
#     drivers_dict[i] = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [22]:
### scraping multiple pages

In [23]:
#sector_urls = get_sector_urls(base_url)
max_workers_num = 30
drivers_dict={}
start_time = time.time()

sector_num = 0
sector_url = sector_urls[sector_num]
sector_url
driver.get(sector_url)

get_num_of_pages_for_sector(sector_num,driver)




#driver.find_elements_by_partial_link_text('Last')[0]

'https://ngodarpan.gov.in/index.php/home/statewise_ngo/171/35/1?per_page=100'

2

In [24]:
#driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
sector_urls = get_sector_urls(base_url)
max_workers_num = 30
drivers_dict={}
start_time = time.time()


sector_num = 4
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers_num) as executor:
    last_sector_page = get_num_of_pages_for_sector(sector_num,driver)
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(last_sector_page,last_sector_page+1))
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(44,45))  \






    
sector_num = 9
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers_num) as executor:
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(106,107))

sector_num = 11
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers_num) as executor:
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(2,3))
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(12,13))
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(15,16))
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(28,29))
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(41,42))
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(64,65))
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(67,68))

sector_num = 12
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers_num) as executor:
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(11,12))
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(17,18))    
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(19,20))    
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(30,31))
    
sector_num = 13
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers_num) as executor:
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(9,10))
    
sector_num = 14
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers_num) as executor:
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(17,18))    

sector_num = 16
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers_num) as executor:
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(79,81))     
    
sector_num = 21
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers_num) as executor:
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(188,189))

sector_num = 28
last_sector_page = get_num_of_pages_for_sector(sector_num,driver)
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers_num) as executor:
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(1,last_sector_page+1))  
    
sector_num = 31
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers_num) as executor:
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(97,98))

sector_num = 34
last_sector_page = get_num_of_pages_for_sector(sector_num,driver)
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers_num) as executor:
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(194,last_sector_page+1))

sector_num = 36
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers_num) as executor:
    results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(104,105))    
    
    
# for sector_num in range(4,37): 
#     last_sector_page = get_num_of_pages_for_sector(sector_num,driver)
#     with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers_num) as executor:
#           results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(last_sector_page,last_sector_page+1))
    
# sector_num = 4
# last_sector_page = get_num_of_pages_for_sector(sector_num,driver)
# with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers_num) as executor:
#       results = executor.map(lambda i: scrape_multiple_pages(sector_num, i, i), range(1,last_sector_page+1))

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


====== WebDriver manager ======
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

====== WebDriver manager ======


Scraping page number 44
Scraping https://ngodarpan.gov.in/index.php/home/statewise_ngo/4794/10/44?per_page=100
Scraping page number 48
Scraping https://ngodarpan.gov.in/index.php/home/statewise_ngo/4794/10/48?per_page=100


Current google-chrome version is 100.0.4896
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\asanc\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
Driver [C:\Users\asanc\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


scraping NGO number 1
scraping NGO number 1
scraping NGO number 2
scraping NGO number 2
scraping NGO number 3
scraping NGO number 3
scraping NGO number 4
scraping NGO number 4
scraping NGO number 5
scraping NGO number 5
scraping NGO number 6
scraping NGO number 6
scraping NGO number 7
scraping NGO number 7
scraping NGO number 8
scraping NGO number 8
scraping NGO number 9
scraping NGO number 9
scraping NGO number 10
scraping NGO number 10
scraping NGO number 11
scraping NGO number 11
scraping NGO number 12
scraping NGO number 12
scraping NGO number 13
scraping NGO number 13
scraping NGO number 14
scraping NGO number 14
scraping NGO number 15
scraping NGO number 15
scraping NGO number 16
scraping NGO number 16
scraping NGO number 17
scraping NGO number 17
scraping NGO number 18
scraping NGO number 18
scraping NGO number 19
scraping NGO number 19
scraping NGO number 20
scraping NGO number 20
scraping NGO number 21
scraping NGO number 21
scraping NGO number 22
scraping NGO number 22
scrapi


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [11:45<00:00, 705.76s/it]


...



100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [07:42<00:00, 462.35s/it]


scraping NGO number 63
scraping NGO number 63
scraping NGO number 64
scraping NGO number 64
scraping NGO number 65
scraping NGO number 65
scraping NGO number 66
scraping NGO number 66
scraping NGO number 67
scraping NGO number 67
scraping NGO number 68
scraping NGO number 68
scraping NGO number 69
scraping NGO number 69
scraping NGO number 70
scraping NGO number 70
scraping NGO number 71
scraping NGO number 71
scraping NGO number 72
scraping NGO number 72
scraping NGO number 73
scraping NGO number 73
scraping NGO number 74scraping NGO number 74

scraping NGO number 75
scraping NGO number 75
scraping NGO number 76
scraping NGO number 76
scraping NGO number 77
scraping NGO number 77
scraping NGO number 78
scraping NGO number 78
scraping NGO number 79
scraping NGO number 79
scraping NGO number 80
scraping NGO number 80
scraping NGO number 81
scraping NGO number 81
scraping NGO number 82
scraping NGO number 82
scraping NGO number 83
scraping NGO number 83
scraping NGO number 84
scraping NG

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [12:25<00:00, 745.69s/it]

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Scraping page number 104
Scraping https://ngodarpan.gov.in/index.php/home/statewise_ngo/10435/19/104?per_page=100
scraping NGO number 1
scraping NGO number 2
scraping NGO number 3
scraping NGO number 4
scraping NGO number 5
scraping NGO number 6
scraping NGO number 7
scraping NGO number 8
scraping NGO number 9
scraping NGO number 10
scraping NGO number 11
scraping NGO number 12
scraping NGO number 13
scraping NGO number 14
scraping NGO number 15
scraping NGO number 16
scraping NGO number 17
scraping NGO number 18
scraping NGO number 19
scraping NGO number 20
scraping NGO number 21
scraping NGO number 22
scraping NGO number 23
scraping NGO number 24
scraping NGO number 25
scraping NGO number 26
scraping NGO number 27
scraping NGO number 28
scraping NGO number 29
scraping NGO number 30
scraping NGO number 31
scraping NGO number 32
scraping NGO number 33
scraping NGO number 34
scraping NGO number 35
scraping NGO number 36
scraping NGO number 37
scraping NGO number 38
scraping NGO number 3

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [12:37<00:00, 757.43s/it]


In [32]:
# This code closes the webdrivers so that they don't occupy space in memory.
drivers_dict

# for i in range(0,len(drivers_dict)):
#     drivers_dict[list(drivers_dict)[i]].quit();
#     try:
#         drivers_dict[i].execute(Command.STATUS)
#         print(f"Web Driver {i} stil active")
#     except:
#          print(f"Web Driver {i} successfully closed")

{'ThreadPoolExecutor-0_11': <selenium.webdriver.chrome.webdriver.WebDriver (session="5bb1b21b97a8e72252123015de8b37dd")>,
 'ThreadPoolExecutor-0_16': <selenium.webdriver.chrome.webdriver.WebDriver (session="c21dbf8012e6710a10daa370937f069d")>,
 'ThreadPoolExecutor-0_15': <selenium.webdriver.chrome.webdriver.WebDriver (session="a76bb763c5e580d5b99414e0b47e290f")>,
 'ThreadPoolExecutor-0_1': <selenium.webdriver.chrome.webdriver.WebDriver (session="267e42d802d7dc9e94d09762af764b37")>,
 'ThreadPoolExecutor-0_3': <selenium.webdriver.chrome.webdriver.WebDriver (session="5e87ef5ff0bf79acc8d1853d6c4288d1")>,
 'ThreadPoolExecutor-0_9': <selenium.webdriver.chrome.webdriver.WebDriver (session="304dfa7de5e90f56726357dac92db545")>,
 'ThreadPoolExecutor-0_14': <selenium.webdriver.chrome.webdriver.WebDriver (session="3cc341d2148960d459a117e89794b6ca")>,
 'ThreadPoolExecutor-0_19': <selenium.webdriver.chrome.webdriver.WebDriver (session="5e4d5f793462e79c62a07fbdac8228ac")>,
 'ThreadPoolExecutor-0_12':

Web Driver 0 successfully closed
Web Driver 1 successfully closed
Web Driver 2 successfully closed
Web Driver 3 successfully closed
Web Driver 4 successfully closed
Web Driver 5 successfully closed
Web Driver 6 successfully closed
Web Driver 7 successfully closed
Web Driver 8 successfully closed
Web Driver 9 successfully closed
Web Driver 10 successfully closed
Web Driver 11 successfully closed
Web Driver 12 successfully closed
Web Driver 13 successfully closed
Web Driver 14 successfully closed
Web Driver 15 successfully closed
Web Driver 16 successfully closed
Web Driver 17 successfully closed
Web Driver 18 successfully closed
Web Driver 19 successfully closed
Web Driver 20 successfully closed
Web Driver 21 successfully closed
Web Driver 22 successfully closed
Web Driver 23 successfully closed
Web Driver 24 successfully closed
Web Driver 25 successfully closed
Web Driver 26 successfully closed
Web Driver 27 successfully closed
Web Driver 28 successfully closed
Web Driver 29 successful

In [ ]:
print("--- The function took  %s seconds ---" % (time.time() - start_time))

In [ ]:
# start_time = time.time()
# sector_df = Parallel(n_jobs=50, verbose=200)(delayed(scrape_a_sector(0)))

In [ ]:
# print("--- The sector scraper took  %s seconds ---" % (time.thttp://localhost:8888/notebooks/Documents/GitHub/expert-forecasting/code/3_get_policymaker_list/python_india_scraping/prototype_ngo_scraping_parallelized.ipynb#alternative:-scraping-all-pages-in-a-sectorime() - start_time))